In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:

df = pd.read_csv("exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [3]:
X = df.drop('koi_disposition', axis = 1)
X.head()
y = df['koi_disposition']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.80)

#A second split for a Hyperparameter tuning set
X_train, X_hp_train, y_train, y_hp_train = train_test_split(X_train, y_train, random_state=0, train_size = 0.5)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [4]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
logi_minmax_scaler = MinMaxScaler().fit(X_train)

X_train_minmax = logi_minmax_scaler.transform(X_train)
X_hp_train_minmax = logi_minmax_scaler.transform(X_hp_train)
X_test_minmax = logi_minmax_scaler.transform(X_test)


# Train the Model



In [14]:
### ESP - Model performs better without scaling
logi_model = LogisticRegression()
logi_model = logi_model.fit(X_train_minmax,y_train)

print(f"Training Data Score: {logi_model.score(X_train_minmax, y_train)}")
print(f"Testing Data Score: {logi_model.score(X_test_minmax, y_test)}")

Training Data Score: 0.8472818311874106
Testing Data Score: 0.8555078683834049


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [15]:
# Create the GridSearchCV model
### ESP - High cv because scores are very close, and I want to make sure I get a representative average

param_grid = {'C': [500, 1000, 1500, 2000],
              'tol': [0.005, 0.001, 0.0001, 0.00001, 0.000001],
              'max_iter': [500, 1000, 1500, 2000, 2500]}
grid = GridSearchCV(logi_model, param_grid, verbose=3, cv = 10, n_jobs = 3)

In [16]:
# Train the model with GridSearch
grid.fit(X_hp_train_minmax, y_hp_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   13.0s
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done 282 tasks      | elapsed:  5.1min
[Parallel(n_jobs=3)]: Done 506 tasks      | elapsed:  9.4min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed: 14.4min
[Parallel(n_jobs=3)]: Done 1000 out of 1000 | elapsed: 18.8min finished


GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=3,
             param_grid={'C': [500, 1000, 1500, 2000],
                         'max_iter': [500, 1000, 1500, 2000, 2500],
                         'tol': [0.005, 0.001, 0.0001, 1e-05, 1e-06]},
             verbose=3)

In [18]:
print(grid.best_params_)
print(grid.best_score_)


{'C': 2000, 'max_iter': 1000, 'tol': 0.005}
0.8830593958013313


In [5]:
logi_model = LogisticRegression(C = 2000, max_iter = 1000, tol = 0.005)
logi_model = logi_model.fit(X_train_minmax,y_train)
print(f"Training Data Score: {logi_model.score(X_train_minmax, y_train)}")
print(f"Testing Data Score: {logi_model.score(X_test_minmax, y_test)}")

Training Data Score: 0.892346208869814
Testing Data Score: 0.8969957081545065


# Save the Model

In [6]:
with open('pickle_jar/logi_model.pickle', 'wb') as dill:
    pickle.dump(logi_model,dill)
with open('pickle_jar/logi_minmax_scaler.pickle', 'wb') as gherkin:
    pickle.dump(logi_minmax_scaler,gherkin)